In [1]:
import jijmodeling as jm

def build_ip_formulation() -> jm.Problem:
    """Create integer programming formulation for the arc-based flow problem.

    This model aligns with the ZPL (0-based) specification, formulating a
    multi-commodity flow problem with integer scaling and big-M constraints.

    Sets:
        - N = {0..n−1}
        - A = {(i, j) | i ≠ j}
        - T = {(k, i, j) | i ≠ j and k ≠ j}

    Parameters:
        - n (int): Number of nodes.
        - t (ndarray, shape (n, n)): Demand matrix, with zero diagonal.
        - M (int): Big-M constant for capacity constraints.
        - intscale (int): Integer scaling factor.

    Variables:
        - x[i, j] ∈ {0, 1}: Binary arc selection variable.
        - f[k, i, j] ∈ ℤ, 0..intscale·M: Flow of commodity k on arc (i, j).
        - z ∈ ℤ, 0..intscale·M: Global upper bound on flow.

    Objective:
        - Minimize z.

    Constraints:
        - c1: ∀ i ∈ N: Σ_{j ≠ i} x[i, j] = 2 (out-degree = 2).
        - c2: ∀ j ∈ N: Σ_{i ≠ j} x[i, j] = 2 (in-degree = 2).
        - c11: ∀ (k, i), k ≠ i:
                  Σ_{j ≠ i} f[k, j, i] − Σ_{j ≠ i, j ≠ k} f[k, i, j]
                  = t[k, i]·intscale
        - c14: ∀ (k, i, j), i ≠ j, k ≠ j:
                  f[k, i, j] ≤ M·intscale·x[i, j]
        - c100: ∀ (i, j), i ≠ j:
                  Σ_{k ≠ j} f[k, i, j] ≤ z

    Returns:
        jm.Problem: JijModeling problem instance with all variables,
        objective, and constraints defined.
    """
    # ---- Placeholders ----
    n = jm.Placeholder("n")
    t = jm.Placeholder("t", ndim=2)   # (n,n)
    M = jm.Placeholder("M")
    intscale = jm.Placeholder("intscale")

    # ---- Indices ----
    i = jm.Element("i", belong_to=(0, n))
    j = jm.Element("j", belong_to=(0, n))
    k = jm.Element("k", belong_to=(0, n))

    # ---- Vars ----
    x = jm.BinaryVar("x", shape=(n, n), description="arc i->j selected")
    f = jm.IntegerVar(
        "f", shape=(n, n, n),
        lower_bound=0, upper_bound=intscale * M,
        description="flow of commodity k on arc i->j"
    )
    z = jm.IntegerVar("z", lower_bound=0, upper_bound=intscale * M)

    # ---- Problem & Objective ----
    problem = jm.Problem("ip_formulation", sense=jm.ProblemSense.MINIMIZE)
    problem += z

    # c1: ∀ i ∈ N :  Σ_{j ≠ i} x[i,j] = 2
    problem += jm.Constraint(
        "c1_outdeg_eq_2",
        jm.sum([(j, j != i)], x[i, j]) == 2,
        forall=[i]
    )
    
    # c2: ∀ j ∈ N :  Σ_{i ≠ j} x[i,j] = 2
    problem += jm.Constraint(
        "c2_indeg_eq_2",
        jm.sum([(i, i != j)], x[i, j]) == 2,
        forall=[j]
    )

    # c11: flow balance
    problem += jm.Constraint(
        "c11_flow_balance",
        jm.sum([(j, j != i)], f[k, j, i])
        - jm.sum([(j, (j != i) & (j != k))], f[k, i, j])
        == t[k, i] * intscale,
        forall=[k, (i, k != i)]
    )

    # c14: capacity bound
    problem += jm.Constraint(
        "c14_capacity_by_x",
        f[k, i, j] <= M * intscale * x[i, j],
        forall=[k, i, (j, (i != j) & (k != j))]
    )

    # c100: z upper bound on flow
    problem += jm.Constraint(
        "c100_z_upper_bounds_flow",
        jm.sum([(k, k != j)], f[k, i, j]) <= z,
        forall=[i, (j, i != j)]
    )

    return problem


In [2]:
problem = build_ip_formulation()
problem

In [3]:
# ---- 準備資料 ----

def cut_matrix(t: list[list[int]], n: int) -> list[list[int]]:
    if not (1 <= n <= 24):
        raise ValueError("n must be between 1 and 24.")
    return [row[:n] for row in t[:n]]
    
n_val = 6
your_t_0based = [
    [0, 24, 43, 23, 21, 41, 61, 21, 20, 0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 24, 19, 23, 0],
    [0, 0, 0, 0, 21, 18, 39, 23, 0, 0, 40, 19, 64, 19, 17, 64, 80, 0, 0, 22, 24, 18, 19, 0],
    [16, 0, 0, 20, 44, 0, 42, 22, 20, 0, 0, 23, 21, 0, 40, 21, 38, 97, 22, 17, 20, 37, 17, 20],
    [42, 20, 18, 0, 40, 57, 43, 42, 0, 0, 19, 17, 0, 39, 0, 0, 22, 41, 0, 0, 17, 42, 40, 43],
    [0, 0, 0, 40, 0, 83, 60, 0, 44, 0, 37, 60, 0, 40, 0, 0, 40, 19, 0, 39, 41, 17, 0, 0],
    [60, 18, 0, 37, 18, 0, 21, 41, 23, 39, 0, 63, 60, 39, 0, 19, 0, 16, 16, 0, 0, 40, 0, 16],
    [22, 0, 0, 0, 0, 0, 0, 61, 36, 80, 96, 19, 19, 41, 16, 0, 0, 0, 22, 0, 43, 0, 44, 22],
    [0, 0, 20, 19, 17, 20, 40, 0, 0, 60, 61, 0, 20, 62, 20, 0, 0, 38, 0, 0, 0, 0, 24, 22],
    [21, 22, 38, 0, 44, 20, 40, 39, 0, 36, 22, 21, 19, 39, 19, 0, 0, 21, 24, 16, 23, 21, 37, 0],
    [0, 24, 23, 39, 20, 0, 0, 41, 0, 0, 0, 22, 0, 0, 44, 42, 22, 42, 22, 19, 20, 58, 18, 0],
    [60, 57, 0, 0, 16, 0, 16, 37, 0, 0, 0, 44, 63, 0, 18, 0, 17, 18, 0, 0, 0, 100, 24, 23],
    [0, 44, 44, 0, 23, 17, 39, 21, 0, 17, 40, 0, 24, 78, 17, 24, 20, 18, 0, 24, 24, 0, 20, 0],
    [23, 16, 0, 0, 0, 23, 0, 0, 0, 0, 43, 58, 0, 0, 24, 60, 0, 0, 19, 0, 21, 0, 20, 0],
    [44, 20, 0, 19, 21, 0, 39, 19, 0, 0, 0, 39, 22, 0, 0, 64, 24, 22, 0, 39, 0, 43, 42, 16],
    [0, 60, 37, 18, 0, 0, 0, 20, 0, 41, 43, 16, 43, 24, 0, 0, 18, 18, 0, 44, 20, 0, 21, 37],
    [0, 0, 23, 39, 0, 24, 40, 0, 37, 0, 40, 20, 44, 43, 0, 0, 0, 0, 0, 16, 0, 59, 0, 0],
    [0, 42, 0, 0, 23, 24, 38, 19, 36, 0, 20, 60, 57, 0, 23, 40, 0, 0, 0, 16, 42, 0, 23, 0],
    [0, 41, 36, 43, 23, 41, 17, 0, 38, 0, 0, 21, 21, 17, 16, 16, 39, 0, 22, 0, 21, 23, 16, 23],
    [17, 0, 23, 23, 20, 0, 17, 58, 17, 0, 20, 0, 17, 24, 0, 0, 17, 42, 0, 58, 19, 22, 0, 24],
    [42, 0, 16, 0, 43, 0, 24, 36, 0, 16, 24, 41, 41, 0, 24, 0, 0, 0, 0, 0, 56, 38, 63, 19],
    [37, 23, 23, 0, 42, 16, 23, 76, 23, 0, 0, 24, 20, 41, 20, 24, 40, 23, 0, 0, 0, 39, 20, 0],
    [43, 20, 17, 17, 0, 20, 19, 0, 80, 0, 0, 0, 40, 0, 40, 16, 19, 0, 0, 0, 18, 0, 17, 0],
    [18, 20, 44, 40, 21, 18, 0, 20, 0, 0, 16, 24, 0, 0, 19, 18, 0, 17, 23, 0, 23, 44, 0, 42],
    [44, 0, 0, 0, 62, 0, 17, 41, 0, 0, 0, 63, 0, 37, 22, 0, 20, 0, 0, 19, 57, 18, 38, 0],
]

instance_data = {
    "n": n_val,
    "t": cut_matrix(your_t_0based, n_val),
    "M": 1000,
    "intscale": 1000,
}


In [4]:
import ommx_pyscipopt_adapter as scip_ad
problem = build_ip_formulation()
ommx_instance = jm.Interpreter(instance_data).eval_problem(problem)
solution = scip_ad.OMMXPySCIPOptAdapter.solve(ommx_instance)

print(f"objective={solution.objective}, feasible={solution.feasible}")

objective=101000.0, feasible=True
